<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/Keras_TransformationList_BeamSearch_Custom_Recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##pip installs

In [1]:
pip install transformers

     |████████████████████████████████| 3.8 MB 6.8 MB/s 
     |████████████████████████████████| 895 kB 64.0 MB/s 
     |████████████████████████████████| 6.5 MB 59.2 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 596 kB 52.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip3 install textattack[tensorflow]

     |████████████████████████████████| 373 kB 8.5 MB/s 
     |████████████████████████████████| 60 kB 8.7 MB/s 
     |████████████████████████████████| 769 kB 9.3 MB/s 
     |████████████████████████████████| 311 kB 75.9 MB/s 
     |████████████████████████████████| 101 kB 14.6 MB/s 
     |████████████████████████████████| 322 kB 69.3 MB/s 
     |████████████████████████████████| 69 kB 9.2 MB/s 
     |████████████████████████████████| 454.3 MB 16 kB/s 
     |████████████████████████████████| 125 kB 81.2 MB/s 
     |████████████████████████████████| 462 kB 83.0 MB/s 
     |████████████████████████████████| 4.9 MB 68.9 MB/s 
     |████████████████████████████████| 132 kB 69.0 MB/s 
     |████████████████████████████████| 4.0 MB 54.4 MB/s 
     |████████████████████████████████| 14.8 MB 67.0 MB/s 
     |████████████████████████████████| 1.2 MB 69.6 MB/s 
     |████████████████████████████████| 4.9 MB 67.1 MB/s 
     |████████████████████████████████| 4.9 MB 66.7 MB/s 
     |█████████████

In [3]:
pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 7.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
pip install folium==0.2.1

     |████████████████████████████████| 69 kB 4.9 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=17de7347abb41cb5a35c25f1726becf1cf190806b56314f778b2e374ae8a6b59
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


## Imports

In [2]:
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack import Attack
from textattack.constraints.pre_transformation import (
        InputColumnModification,
        MaxModificationRate,
        RepeatModification,
        StopwordModification,
        MaxWordIndexModification
                )
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import BeamSearch
from textattack.transformations import (
   
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapHomoglyphSwap,
    WordSwapEmbedding,
    WordSwapHowNet,
    WordSwapInflections,
    WordDeletion,
    WordSwapGradientBased,
    WordSwapExtend,
    WordSwapContract,
    WordSwapChangeName,
    WordSwapChangeNumber,
    WordSwapChangeLocation,
    WordSwapRandomCharacterSubstitution,
    WordSwapMaskedLM,
    WordInsertionMaskedLM,
    WordMergeMaskedLM,
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe

from abc import ABC, abstractmethod
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from IPython.core.display import display, HTML

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:12<00:00, 37.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpu25rpjzk.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [3]:
from textattack.datasets import HuggingFaceDataset
from textattack.attacker import Attacker
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.goal_functions import TargetedClassification
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
import torch

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb")
# We wrap the model so it can be used by textattack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

#dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
dataset = HuggingFaceDataset("imdb", split="test")
#dataset = HuggingFaceDataset("glue", "sst2", "train")

textattack: Loading datasets dataset imdb, split test.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.


##Custom Recipes

##Recipe1    
            WordSwapQWERTY
            WordSwapWordNet
            WordSwapHomoglyphSwap
            WordSwapNeighboringCharacterSwap
            WordSwapRandomCharacterInsertion
            WordSwapRandomCharacterDeletion
            WordSwapEmbedding
      

In [4]:
class Recipe(Attack,ABC):
    @staticmethod
    def build(model_wrapper,transf):
        if(transf == "WordSwapWordNet"):
            transformation = WordSwapWordNet()
        elif(transf == "WordSwapQWERTY"):
            transformation = WordSwapQWERTY()
        elif(transf == "WordSwapHomoglyphSwap"):
            transformation = WordSwapHomoglyphSwap()
        elif(transf == "WordSwapNeighboringCharacterSwap"):
            transformation = WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        elif(transf == "WordSwapRandomCharacterInsertion"):
            transformation = WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    )
        elif(transf == "WordSwapRandomCharacterDeletion"):
             transformation = WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    )
        elif(transf == "WordSwapEmbedding"):
             transformation = WordSwapEmbedding(max_candidates=5)
        elif(transf == "WordSwapHowNet"):
             transformation = WordSwapHowNet()
        elif(transf == "WordSwapInflections"):
             transformation = WordSwapInflections()
        elif(transf == "WordDeletion"):
             transformation = WordDeletion()
        elif(transf == "WordSwapGradientBased"):
             transformation = WordSwapGradientBased(model_wrapper, top_n=1)
        elif(transf == "WordSwapExtend"):
             transformation = WordSwapExtend()
        elif(transf == "WordSwapContract"):
             transformation = WordSwapContract()
        elif(transf == "WordSwapChangeName"):
             transformation = WordSwapChangeName()
        elif(transf == "WordSwapChangeNumber"):
             transformation = WordSwapChangeNumber()  
        elif(transf == "WordSwapChangeLocation"):
             transformation = WordSwapChangeLocation()
        elif(transf == "WordSwapRandomCharacterSubstitution"):
             transformation = WordSwapRandomCharacterSubstitution()
        elif(transf == "WordSwapMaskedLM"):
             transformation = WordSwapMaskedLM(method="bert-attack", max_candidates=48)
        elif(transf == "WordInsertionMaskedLM"):
             transformation = WordInsertionMaskedLM(
                    #masked_language_model=shared_masked_lm,
                    #tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=0.0,
                )
        elif(transf == "WordMergeMaskedLM"):
             transformation = WordMergeMaskedLM(
                    #masked_language_model=shared_masked_lm,
                    #tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=5e-3,
                )

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = BeamSearch(beam_width = 8)
        return Attack(goal_function, constraints, transformation, search_method)


'WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution'

In [ ]:
logger = CSVLogger(color_method='html')
transf_list = ['WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
               'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
               'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution','WordSwapMaskedLM',
               'WordInsertionMaskedLM','WordMergeMaskedLM']
              
          
for transf in transf_list:
    print("Starting attack for-----------------",transf)
    attack = Recipe.build(model_wrapper,transf)
    attack_args = AttackArgs(num_examples=8,)
    attacker = Attacker(attack, dataset,attack_args)
    attack_results = attacker.attack_dataset()
    for result in attack_results:
        logger.log_attack_result(result)
    display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


Starting attack for----------------- WordSwapWordNet


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/8 [00:00<?, ?it/s]

Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  12%|█▎        | 1/8 [02:42<18:59, 162.73s/it]

...

Starting attack for----------------- WordSwapMaskedLM


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/8 [00:00<?, ?it/s]

Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  48
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  12%|█▎        | 1/8 [03:06<21:45, 186.45s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (99%)]]

I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was [[great]], and I suggest that you go see it before you judge.

I went and saw this movie last night after being coaxed

In [ ]:
#logger = CSVLogger(color_method='html')
transf_list = ['WordMergeMaskedLM','WordSwapExtend','WordSwapContract',
              'WordSwapChangeName','WordSwapChangeNumber','WordSwapChangeLocation','WordSwapMaskedLM','WordInsertionMaskedLM']
              
          
for transf in transf_list:
    print("Starting attack for-----------------",transf)
    attack = Recipe.build(model_wrapper,transf)
    attack_args = AttackArgs(num_examples=8)
    attacker = Attacker(attack, dataset,attack_args)
    attack_results = attacker.attack_dataset()
    #for result in attack_results:
    #    logger.log_attack_result(result)
    #display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

Starting attack for----------------- WordMergeMaskedLM


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/8 [00:00<?, ?it/s]

Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordMergeMaskedLM(
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.005
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

2022-03-07 04:46:11,108 --------------------------------------------------------------------------------
2022-03-07 04:46:11,109 The model key 'upos-fast' now maps to 'https://huggingface.co/flair/upos-english-fast' on the HuggingFace ModelHub
2022-03-07 04:46:11,110  - The most current version of the model is automatically downloaded from there.
2022-03-07 04:46:11,111  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/upos-fast/en-upos-ontonotes-fast-v0.4.pt)
2022-03-07 04:46:11,113 --------------------------------------------------------------------


2022-03-07 04:46:13,509 loading file /root/.flair/models/upos-english-fast/b631371788604e95f27b6567fe7220e4a7e8d03201f3d862e6204dbf90f9f164.0afb95b43b32509bf4fcc3687f7c64157d8880d08f813124c1bd371c3d8ee3f7


  0%|          | 0/8 [12:21<?, ?it/s]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  12%|█▎        | 1/8 [00:18<02:11, 18.78s/it]

...


Starting attack for----------------- WordSwapMaskedLM


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/8 [00:00<?, ?it/s]

Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  48
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  12%|█▎        | 1/8 [03:04<21:29, 184.22s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (99%)]]

I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was [[great]], and I suggest that you go see it before you judge.

I went and saw this movie last night after being coaxed